# Video Data EM Notebook

In [70]:
import sys
#sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os
import time

In [71]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

In [72]:
# Get the paths
path_A = os.getcwd() + '/DATA/imdb3.csv'
path_B = os.getcwd() + '/DATA/thenumbers3.csv'
print(path_A)

/Users/dcompgriff/PrivateDocs/College Docs/Graduate School/University of Wisconson/School Docs/Spring 2017/839 Data Analysis/Project Stage 3/DATA/imdb3.csv


In [73]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
A.head()

,id,title,year,mpaa,runtime,genres,director,stars,gross
0,0,Toy Story 2,1999,G,92 min,"Animation, Adventure, Comedy",John Lasseter,"Ash Brannon,Lee Unkrich,Tom Hanks,Tim Allen,Joan Cusack,Kelsey Grammer",$245.85M
1,1,The Outlaw Josey Wales,1976,PG,135 min,Western,Clint Eastwood,"Clint Eastwood,Sondra Locke,Chief Dan George,Bill McKinney",$31.80M
2,2,"Monsters, Inc.",2001,G,92 min,"Animation, Adventure, Comedy",Pete Docter,"David Silverman,Lee Unkrich,Billy Crystal,John Goodman,Mary Gibbs,Steve Buscemi",$289.92M
3,3,In the Heat of the Night,1967,Not Rated,110 min,"Crime, Drama, Mystery",Norman Jewison,"Sidney Poitier,Rod Steiger,Warren Oates,Lee Grant",$24.38M
4,4,Chungking Express,1994,PG-13,102 min,"Crime, Drama, Romance",Kar-Wai Wong,"Brigitte Lin,Takeshi Kaneshiro,Tony Chiu-Wai Leung,Faye Wong",$0.60M


In [74]:
B.head()

,id,title,year,mpaa,runtime,genres,director,stars,gross
0,0,Effects,2005,NaN,NaN,NaN,Dusty Nelson,NaN,NaN
1,1,Ek Haseena Thi Ek Deewana Tha,2017,NaN,105 minutes,Drama,Suneel Darshan,NaN,"$149,491"
2,2,Ekk Albela,2016,NaN,NaN,Drama,Shekhar Sartandel,NaN,"$1,907"
3,3,The Daisy Chain,2009,R,NaN,NaN,NaN,NaN,NaN
4,4,Dali & I: The Surreal Story,2011,NaN,NaN,Drama,Andrew Niccol,Al Pacino,NaN


### Check for null entries

In [75]:
B.isnull().sum()

id              0
title           0
year            0
mpaa        12849
runtime     16729
genres       5146
director    12462
stars       22044
gross       19798
dtype: int64

In [76]:
A.isnull().sum()

id             0
title          0
year           0
mpaa          51
runtime        1
genres         0
director       0
stars          4
gross       1239
dtype: int64

### Check the data size

In [77]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 4291
Number of tuples in B: 31006
Number of tuples in A X B (i.e the cartesian product): 133046746


In [78]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [79]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 200, 1, show_progress=False)
len(A1), len(B1)

# But for the purposes of this notebook, we will use the entire table A and B

(133, 200)

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [80]:
# Blocking plan

# A, B -- overlap blocker [title] --------------------|---> candidate set

In [81]:
# Create attribute equivalence blocker
ab = em.OverlapBlocker()

# Block using city attribute
C1 = ab.block_tables(A, B, 'title', 'title', 
                    l_output_attrs=['title', 'year', 'mpaa', 'runtime', 'genres', 'director', 'stars', 'gross'], 
                    r_output_attrs=['title', 'year', 'mpaa', 'runtime', 'genres', 'director', 'stars', 'gross'],
                    overlap_size=1
                    )

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:50


In [82]:
len(C1)

10956134

## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10,956,134 (from 133,046,746), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [83]:
# Debug blocker output
startTime = time.time()
dbg = em.debug_blocker(C1, A, B, output_size=200, attr_corres=[('title','title'), ('year', 'year')])
endTime = time.time()
print("Total time: %.2f seconds."%(endTime-startTime))

Total time: 43.82 seconds.


In [84]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head(50)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,rtable_title,rtable_year
0,0,2965,17445,Advise & Consent,1962,Ginger & Rosa,2013
1,1,2965,17842,Advise & Consent,1962,Harry & Son,1984
2,2,2965,17844,Advise & Consent,1962,Harry & Snowman,2016
3,3,2965,19318,Advise & Consent,1962,Lambert & Stamp,2015
4,4,2965,19600,Advise & Consent,1962,David & Layla,2007
5,5,2965,19796,Advise & Consent,1962,Hansel & Gretel,2002
6,6,2965,20161,Advise & Consent,1962,Q & A,1990
7,7,2965,23293,Advise & Consent,1962,Starsky & Hutch,2004
8,8,2965,23439,Advise & Consent,1962,Spring & Arnaud,2016
9,9,2965,25285,Advise & Consent,1962,Town & Country,2001


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [88]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--

In [99]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C2 = ob.block_candset(C1, 'year', 'year' 
                    )
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:28


134228

In [103]:
# Display first two rows from C2
C2.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
34,34,45,3,The Hangover,2009,R,100 min,Comedy,Todd Phillips,"Zach Galifianakis,Bradley Cooper,Justin Bartha,Ed Helms",$277.32M,The Daisy Chain,2009,R,NaN,NaN,NaN,NaN,NaN
144,144,4286,3,The Loved Ones,2009,R,84 min,"Horror, Thriller",Sean Byrne,"Xavier Samuel,Robin McLeavy,Victoria Thaine,Jessica McNamee",NaN,The Daisy Chain,2009,R,NaN,NaN,NaN,NaN,NaN


In [105]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--
#
# C2 ------ overlap blocker [mpaa] --------> C3--

In [106]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C3 = ob.block_candset(C2, 'mpaa', 'mpaa' 
                    )
len(C3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


27581

In [107]:
# Display first two rows from C3
C3.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
34,34,45,3,The Hangover,2009,R,100 min,Comedy,Todd Phillips,"Zach Galifianakis,Bradley Cooper,Justin Bartha,Ed Helms",$277.32M,The Daisy Chain,2009,R,NaN,NaN,NaN,NaN,NaN
144,144,4286,3,The Loved Ones,2009,R,84 min,"Horror, Thriller",Sean Byrne,"Xavier Samuel,Robin McLeavy,Victoria Thaine,Jessica McNamee",NaN,The Daisy Chain,2009,R,NaN,NaN,NaN,NaN,NaN


We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [20]:
# Debug again
dbg = em.debug_blocker(C, A, B)

In [21]:
# Display first few rows from the debugger output
dbg.head(3)

,_id,ltable_id,rtable_id,ltable_name,ltable_addr,ltable_city,ltable_phone,ltable_type,rtable_name,rtable_addr,rtable_city,rtable_phone,rtable_type
0,0,860,254,pen & pencil,205 e. 45th st.,new york,212/682-8660,american,gotham bar & grill,12 e. 12th st.,new york city,212-620-4020,american (new)
1,1,883,264,shaan,57 w. 48th st.,new york,212/ 977-8400,asian,manhattan ocean club,57 w. 58th st.,new york city,212-371-7777,seafood
2,2,808,256,jewel of india,15 w. 44th st.,new york,212/869-5544,asian,island spice,402 w. 44th st.,new york city,212-765-1737,caribbean


We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [22]:
# Sample  candidate set
S = em.sample_table(C, 450)

Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [23]:
# Label S
G = em.label_table(S, 'gold')

/anaconda2/lib/python2.7/site-packages/py_entitymatching/gui/table_gui.py:94: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  table.set_value(idxv[i], cols[j], val)


For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [24]:
# Load the pre-labeled data
path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(G)

450

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [25]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [26]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [27]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [28]:
# List the names of the features generated
feature_table['feature_name']

0                         id_id_exm
1                         id_id_anm
2                    id_id_lev_dist
3                     id_id_lev_sim
4         name_name_jac_qgm_3_qgm_3
5     name_name_cos_dlm_dc0_dlm_dc0
6     name_name_jac_dlm_dc0_dlm_dc0
7                     name_name_mel
8                name_name_lev_dist
9                 name_name_lev_sim
10                    name_name_nmw
11                     name_name_sw
12        addr_addr_jac_qgm_3_qgm_3
13    addr_addr_cos_dlm_dc0_dlm_dc0
14    addr_addr_jac_dlm_dc0_dlm_dc0
15                    addr_addr_mel
16               addr_addr_lev_dist
17                addr_addr_lev_sim
18                    addr_addr_nmw
19                     addr_addr_sw
20        city_city_jac_qgm_3_qgm_3
21    city_city_cos_dlm_dc0_dlm_dc0
22    city_city_jac_dlm_dc0_dlm_dc0
23                    city_city_mel
24               city_city_lev_dist
25                city_city_lev_sim
26                    city_city_nmw
27                     city_

### Converting the development set to feature vectors

In [29]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)

In [30]:
# Display first few rows
H.head(3)

,_id,ltable_id,rtable_id,id_id_exm,id_id_anm,id_id_lev_dist,id_id_lev_sim,name_name_jac_qgm_3_qgm_3,name_name_cos_dlm_dc0_dlm_dc0,name_name_jac_dlm_dc0_dlm_dc0,...,city_city_sw,type_type_jac_qgm_3_qgm_3,type_type_cos_dlm_dc0_dlm_dc0,type_type_jac_dlm_dc0_dlm_dc0,type_type_mel,type_type_lev_dist,type_type_lev_sim,type_type_nmw,type_type_sw,gold
221,1790,563,248,0,0.440497,3,0.0,1.000000,1.000000,1.000000,...,8.0,0.235294,0.0,0.0,0.883333,7.0,0.416667,-2.0,4.0,1
439,794,544,116,0,0.213235,3,0.0,0.258065,0.500000,0.333333,...,1.0,0.000000,0.0,0.0,0.466667,4.0,0.200000,0.0,1.0,0
191,2315,589,305,0,0.517827,3,0.0,0.172414,0.353553,0.200000,...,1.0,0.000000,0.0,0.0,0.451923,24.0,0.000000,-11.0,2.0,0


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [31]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/anaconda2/lib/pyth

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.976471,0.936450,0.954078
1,RF,0.975735,0.963235,0.968450
2,SVM,1.000000,0.297437,0.440952
3,LinReg,1.000000,0.975735,0.987488
4,LogReg,0.974603,0.948214,0.960212
5,NaiveBayes,0.953947,0.973214,0.962530


### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [32]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [33]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        target_attr='gold')

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

In [34]:
def phone_phone_feature(ltuple, rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    p1 = p1.replace('/','-')
    p1 = p1.replace(' ','')
    p2 = p2.replace('/','-')
    p2 = p2.replace(' ','')    
    if p1 == p2:
        return 1.0
    else:
        return 0.0

In [35]:
feature_table = em.get_features_for_matching(A, B)
em.add_blackbox_feature(feature_table, 'phone_phone_feature', phone_phone_feature)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,id,id,numeric,numeric,Exact Match; Absolute Norm
1,name,name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,addr,addr,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
3,city,city,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,phone,phone,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
5,type,type,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


KeyboardInterrupt: 

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [ ]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='gold', show_progress=False)

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

Now, observe the best matcher is achieving a better F1. Let us stop here and  proceed on to evaluating the best matcher on the unseen data (the evaluation set).

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)